# Automatización de un proceso en Excel

## Procesamiento de un archivo

Empezaremos procesando un archivo Excel para tener un punto de inicio en el algoritmo de automatización.\
Si podemos procesar un archivo Excel en Python, entonces también podemos procesar una gran cantidad de archivos de la misma manera.

Representaremos el archivo Excel dentro de Python a través de un objeto llamado `cuadro`:

In [ ]:
import pandas as pd

In [ ]:
cuadro = pd.read_excel("files/excel/Proyecto Python DH-01.xlsx", header=12, usecols="c:s")
cuadro.rename(columns=dict(zip(list(cuadro.columns)[-2:], ["RMR", "Calidad RMR"])), inplace=True)

In [ ]:
cuadro.head()

In [ ]:
cuadro["Longitud de corrida (m)"] = cuadro["Hasta (m)"] - cuadro["Desde (m)"]
cuadro["Recuperación de taladro (%)"] = 100 * cuadro["Recuperación de taladro (m)"] / cuadro["Longitud de corrida (m)"]
cuadro["RQD (%)"] = 100 * cuadro["RQD (m)"] / cuadro["Longitud de corrida (m)"]
cuadro["Frecuencia de fracturas (FF/m)"] = cuadro["Fracturas por corrida"] / cuadro["Longitud de corrida (m)"]

In [ ]:
cuadro.head()

## Algoritmo para calcular el RMR

Crearemos funciones que nos permitan calcular la puntuación RMR y la calidad RMR.

In [ ]:
# RQD
def rqd(dato):
    if 90 < dato:
        return 20
    elif 75 < dato <= 90:
        return 17
    elif 50 < dato <= 75:
        return 13
    elif 25 < dato <= 50:
        return 8
    elif dato <= 25:
        return 3

# Espaciamiento
def espaciamiento(dato):
    if dato > 2000:
        return 20
    elif 600 < dato <= 2000:
        return 15
    elif 200 < dato <= 600:
        return 10
    elif 60 < dato <= 200:
        return 8
    elif dato <= 60:
        return 5    

# Resistencia
def resistencia(dato):
    if dato == "R6":
        return 15
    elif dato == "R5":
        return 12
    elif dato == "R4":
        return 7
    elif dato == "R3":
        return 4
    elif dato == "R2":
        return 2
    elif dato == "R1":
        return 1
    elif dato == "R0":
        return 0    
    
# Apertura
def apertura(dato):
    if dato == "A0":
        return 6
    elif dato == "A1":
        return 5
    elif dato == "A2":
        return 3
    elif dato == "A3":
        return 1
    elif dato == "A4":
        return 0    

# Rugosidad
def rugosidad(dato):
    if dato == "G1":
        return 6
    elif dato == "G2":
        return 5
    elif dato == "G3":
        return 3
    elif dato == "G4":
        return 1
    elif dato == "G5":
        return 0
    
# Relleno
def relleno(dato):
    if dato == "F0":
        return 6
    elif dato == "F1":
        return 4
    elif dato == "F2":
        return 2
    elif dato == "F3":
        return 1
    elif dato == "F4":
        return 0    
    
# Alteración
def alteracion(dato):
    if dato == "W0":
        return 6
    elif dato == "W1":
        return 5
    elif dato == "W2":
        return 3
    elif dato == "W3":
        return 1
    elif dato == "W4":
        return 0  
    
# Persistencia (promedio de apertura, rugosidad, relleno y alteración)

# Agua = 15

# Calidad RMR
def calidad(dato):
    if 80 < dato:
        return "Muy buena"
    elif 60 < dato <= 80:
        return "Buena"
    elif 40 < dato <= 60:
        return "Media"
    elif 20 < dato <= 40:
        return "Mala"
    elif dato <= 20:
        return "Muy mala"

Usaremos estas funciones dentro del cuadro para observar el resultado.

In [ ]:
RQD = cuadro["RQD (%)"].apply(rqd)
E = cuadro["Espaciamiento (mm)"].apply(espaciamiento)
R = cuadro["Resistencia a la compresión simple"].apply(resistencia)
A = cuadro["Apertura"].apply(apertura)
G = cuadro["Rugosidad"].apply(rugosidad)
F = cuadro["Relleno"].apply(relleno)
W = cuadro["Alteración"].apply(alteracion)
P = (A + G + F + W) / 4

RMR = RQD + E + R + A + G + F + W + P + 15

cuadro["RMR"] = RMR
cuadro["Calidad RMR"] = cuadro["RMR"].apply(calidad)

In [ ]:
cuadro.head(10)

Este es el resultado usando solamente un archivo de Excel, para automatizar este proceso debemos expandir el algoritmo y aplicarlo en una lista de archivos.

## Algoritmo de automatización

Empezaremos cambiando la ruta de trabajo por la dirección de la carpeta donde tenemos los archivos.

In [ ]:
import pandas as pd
import os
import xlwings as xw
from tqdm import tqdm

os.chdir("D:\Desktop\python_excel")

carpeta = os.listdir()[:-1]

carpeta

Luego, usando un **bucle**, procesaremos la información dentro de Python y guardaremos el resultado de cada archivo en Excel.

In [ ]:
app = xw.App(visible=False, add_book=False)

for archivo in tqdm(carpeta):
    # Crear el cuadro con pandas
    cuadro = pd.read_excel(archivo, header=12, usecols="c:s")
    cuadro.rename(columns=dict(zip(list(cuadro.columns)[-2:], ["RMR", "Calidad RMR"])), inplace=True)
    cuadro["Longitud de corrida (m)"] = cuadro["Hasta (m)"] - cuadro["Desde (m)"]
    cuadro["Recuperación de taladro (%)"] = 100 * cuadro["Recuperación de taladro (m)"] / cuadro["Longitud de corrida (m)"]
    cuadro["RQD (%)"] = 100 * cuadro["RQD (m)"] / cuadro["Longitud de corrida (m)"]
    cuadro["Frecuencia de fracturas (FF/m)"] = cuadro["Fracturas por corrida"] / cuadro["Longitud de corrida (m)"]
    
    RQD = cuadro["RQD (%)"].apply(rqd)
    E = cuadro["Espaciamiento (mm)"].apply(espaciamiento)
    R = cuadro["Resistencia a la compresión simple"].apply(resistencia)
    A = cuadro["Apertura"].apply(apertura)
    G = cuadro["Rugosidad"].apply(rugosidad)
    F = cuadro["Relleno"].apply(relleno)
    W = cuadro["Alteración"].apply(alteracion)
    P = (A + G + F + W) / 4
    RMR = RQD + E + R + A + G + F + W + P + 15
    cuadro["RMR"] = RMR
    cuadro["Calidad RMR"] = cuadro["RMR"].apply(calidad)
    
    # Escribir el Dataframe de Pandas en Excel
    data = app.books.open(archivo)
    hoja = data.sheets[0]
    hoja["c13"].options(pd.DataFrame, index=False, transpose=False).value = cuadro
    data.save()   # Guardamos el archivo
    data.close()  # Cerramos el archivo

app.quit()    